In [ ]:
# 5) Read prompts exactly as before
data = []
with open("/kaggle/input/prompt-for-vlsp2020/output.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        utt, text = line.split("|", 1)
        data.append({"audio": utt, "sentence": text.lower()})

In [ ]:
data_test = []
with open("/kaggle/input/vivos-vietnamese/vivos/test/prompts.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        utt, text = line.split(" ", 1)

        speaker = utt.split("_")[0]
        wav     = "/kaggle/input/vivos-vietnamese/vivos/test/waves/" + speaker + f"/{utt}.wav"
        data_test.append({"audio": str(wav), "sentence": text.lower()})

In [ ]:
from datasets import Dataset, Audio, DatasetDict

dataset = DatasetDict()

# large_dataset = Dataset.from_list(data).cast_column("audio", Audio(sampling_rate=16000))

# vivos["train"] = dataset["train"].shuffle(seed=42).select(range(100))
# vivos["test"] = dataset["test"].shuffle(seed=42).select(range(20))
# current: 30001, 
#:15000, 15001:30000, 30001:45000, 45001:56426
dataset["test"] = Dataset.from_list(data_test).cast_column("audio", Audio(sampling_rate=16000))
dataset["train"] = Dataset.from_list(data[30001:45000]).cast_column("audio", Audio(sampling_rate=16000))

print(dataset)

In [ ]:
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer

model_name_or_path = "openai/whisper-small"
task = "transcribe"
language = "Vietnamese"
language_abbr = "vi"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        # cắt nếu token vượt quá 448
        if labels.size(1) >= 448:
            labels = labels[:, :447]

        batch["labels"] = labels

        return batch

In [ ]:
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    print(wer)

    return {"wer": wer}

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration

peft_model_id = "habuiphuc/Whisper-tiny-finetun" # Use the same model ID as before.
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id, is_trainable=True)

model.generation_config.language = language
model.generation_config.task = task
model.generation_config.forced_decoder_ids = None

In [ ]:
model.print_trainable_parameters()


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="habuiphuc/Whisper-tiny-finetun",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    # eval_steps=1875,
    fp16=True,
    predict_with_generate=True,
    report_to=["tensorboard"],
    #report_to="none",
    per_device_eval_batch_size=8,
    generation_max_length=254,
    logging_steps=100,
    # save_steps=1875,        # Lưu mỗi 500 bước
    save_total_limit=3,
    # max_steps=2000, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
    push_to_hub=True,
    # load_best_model_at_end=True,
    # metric_for_best_model="wer",
    # greater_is_better=False,
)

In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()


In [ ]:
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

eval_dataloader = DataLoader(dataset["train"].shuffle(seed=42).select(range(700)), batch_size=8, collate_fn=data_collator)

forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []


model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch
    gc.collect()

wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer on train": wer, "eval/normalized_wer on train": normalized_wer}

print(eval_metrics)

In [ ]:
trainer.push_to_hub()
